In [ ]:
from astroquery.mast import Observations
from astropy.io import fits
import drizzlepac
import glob
import os

In [ ]:
files_15113 = glob.glob('/astro/4/calamida/lco/15113/*drz.fits')
files_13711 = glob.glob('/astro/4/calamida/lco/13711/*drz.fits')
file_names_15113 = [os.path.basename(x).replace('_drz.fits', '') for x in files_15113]
file_names_13711 = [os.path.basename(x).replace('_drz.fits', '') for x in files_13711]
all_file_names = file_names_15113 + file_names_13711

len(glob.glob('/astro/4/calamida/lco/15113/*drz.fits') + glob.glob('/astro/4/calamida/lco/13711/*drz.fits'))


In [ ]:
all_file_names

In [ ]:
from astroquery.mast import Observations

# you can see all available filters for query_criteria in Observations.get_metadata('observations')
obs_table = Observations.query_criteria(project='JWST', proposal_id='02736', filters=['F200W'])

# the above will get you the table of all dataset ids (asn level) matching that criteria, so to
# get each product under each ID do:
prods = Observations.get_product_list(obs_table)

# you can further filter the list above, for example if you want uncal files only. note that here
# there is a field 'mrp_only' which is False by default unlike the MAST portal. case sensitive.
prods = Observations.filter_products(prods, productSubGroupDescription=['UNCAL'])

# when you are happy with 'prods', download everything in that table by doing:
#manifest = Observations.download_products(prods, productType="SCIENCE")



In [ ]:
prods

In [ ]:
for f in files_15113:
    fname = os.path.basename(f).replace('_drz.fits', '')[0:8] + '*'
    obs_table = Observations.query_criteria(proposal_id='15113', obs_id=fname, project='HST')
    prods = Observations.get_product_list(obs_table)
    prods = Observations.filter_products(prods, productSubGroupDescription=['FLT'], type='S')
    print(obs_table['target_name'])
    #manifest = Observations.download_products(prods, productType="SCIENCE")

In [ ]:
for f in files_13711:
    fname = os.path.basename(f).replace('_drz.fits', '')[0:8] + '*'
    obs_table = Observations.query_criteria(proposal_id='13711', obs_id=fname, project='HST')
    prods = Observations.get_product_list(obs_table)
    prods = Observations.filter_products(prods, productSubGroupDescription=['FLT'], type='S')
    print(obs_table['target_name'])
    #manifest = Observations.download_products(prods, productType="SCIENCE")

In [ ]:
flts = glob.glob('mastDownload/HST/*/*flt.fits')
flts

In [ ]:
asns = {}
for f in flts:
    asn = fits.getval(f, 'ASN_ID')
    if asn not in asns.keys():
        asns[asn] = [f]
    else:
        asns[asn].append(f)
_ = asns.pop('ICMDP90J0')
_ = asns.pop('icmdr00g0'.upper())

asns['NONE'].append('mastDownload/HST/icmdp9b2q/icmdp9b2q_flt.fits')
asns['NONE'].append('mastDownload/HST/icmdr0ctq/icmdr0ctq_flt.fits')
single_files = asns.pop('NONE')


In [ ]:
asns

In [ ]:
for i, val in enumerate(list(asns.keys())):
    print('RUNNING ON ', list(asns.keys())[i].lower())
    print('FLTS = ', asns[list(asns.keys())[i]])
    
    drizzlepac.astrodrizzle.AstroDrizzle(input=asns[list(asns.keys())[i]],
                                         driz_sep_rot=0.0, mdriztab=True,
                                         output=list(asns.keys())[i].lower(),
                                         build=True, in_memory=True, clean=True)
    print('OUTPUT FILE NAME =', list(asns.keys())[i].lower())
    print('-------------------------')

In [ ]:
single_files

for f in single_files:
    print(f, fits.getval(f, 'targname'))
    drizzlepac.astrodrizzle.AstroDrizzle(input=f,
                                         driz_sep_rot=0.0, mdriztab=True,
                                         output=os.path.basename(f).replace('_flt.fits', ''),
                                         build=True, in_memory=True, clean=True)
    

In [ ]:
drizzlepac.astrodrizzle.AstroDrizzle(input='mastDownload/HST/icmdp9b2q/icmdp9b2q_flt.fits',
                                         driz_sep_rot=0.0, mdriztab=True,
                                         output='icmdp9b2q',
                                         build=True, in_memory=True, clean=True)

In [ ]:
!rm idgq030l0_drz.fits

In [ ]:
! rm icmdr00g0_drz.fits 

In [ ]:
drizzlepac.astrodrizzle.AstroDrizzle(input='mastDownload/HST/icmdr0ctq/icmdr0ctq_flt.fits',
                                         driz_sep_rot=0.0, mdriztab=True,
                                         output='icmdr0ctq',
                                         build=True, in_memory=True, clean=True)

In [ ]:
for f in flts:
    print(fits.getval(f, 'targname'))

In [ ]:
fits.open('mastDownload/HST/icmdp9b2q/icmdp9b2q_flt.fits')[0].header['targname']